# 라이브러리 및 파일 호출

In [1]:
#라이브러리 호출

import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats as spst
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8")  # matplotlib style 설정

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [313]:
#csv 파일 호출

df_orders=pd.read_csv("./data file/olist_orders.csv")
df_customers=pd.read_csv('./data file/olist_customers.csv')
df_order_items=pd.read_csv('./data file/olist_order_items.csv')
df_order_payments=pd.read_csv('./data file/olist_order_payments.csv')
df_products=pd.read_csv('./data file/olist_products.csv')
df_trans=pd.read_csv('./data file/product_category_name_translation.csv')

In [314]:
#전체 병합 파일 df 생성

df_co = pd.merge(df_customers, df_orders, on='customer_id', how='left')
df_co['total_orders'] = df_co.groupby('customer_unique_id')['order_id'].transform('count')
df_coi = pd.merge(df_co, df_order_items, on="order_id", how='left')
df_coi['total_order_items'] = df_coi.groupby('customer_unique_id')['customer_unique_id'].transform('count')
df_coip = pd.merge(df_coi, df_order_payments, on="order_id", how="left")
df_product=pd.merge(df_products,df_trans,on='product_category_name',how='outer')
mask = ~df_product['product_category_name_english'].isnull()
df_product.loc[mask, 'product_category_name'] = df_product.loc[mask, 'product_category_name_english']
df = pd.merge(df_coip, df_product, on="product_id", how="left")

# 데이터 전처리

## 재구매 여부 열 생성

In [315]:
df.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'order_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'total_orders', 'product_id',
       'seller_id', 'price', 'freight_value', 'total_order_items',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'product_category_name', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'product_category_name_english'],
      dtype='object')

In [316]:
df['is_once']= df['total_orders'].apply(lambda x: 1 if x==1 else 0)
df  

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,payment_type,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,is_once
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,00e7ee1b050b8499577073aeb2a297a1,delivered,2022-05-16 15:05:35,2022-05-16 15:22:12,2022-05-25 10:35:35,2022-06-05 00:00:00,1,...,credit_card,2.0,146.87,office_furniture,8683.0,54.0,64.0,31.0,office_furniture,1
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,29150127e6685892b6eab3eec79f59c7,delivered,2023-01-12 20:48:24,2023-01-12 20:58:32,2023-01-29 12:41:19,2023-02-06 00:00:00,1,...,credit_card,8.0,335.48,housewares,10150.0,89.0,15.0,40.0,housewares,1
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2023-05-19 16:07:45,2023-05-20 16:19:10,2023-06-14 17:58:51,2023-06-13 00:00:00,1,...,credit_card,7.0,157.73,office_furniture,8267.0,52.0,52.0,17.0,office_furniture,1
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,951670f92359f4fe4a63112aa7306eba,delivered,2023-03-13 16:06:38,2023-03-13 17:29:19,2023-03-28 16:04:25,2023-04-10 00:00:00,1,...,credit_card,1.0,173.30,office_furniture,12160.0,56.0,51.0,28.0,office_furniture,1
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2023-07-29 09:51:30,2023-07-29 10:10:09,2023-08-09 20:55:48,2023-08-15 00:00:00,1,...,credit_card,8.0,252.25,home_confort,5200.0,45.0,15.0,35.0,home_confort,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,6760e20addcf0121e9d58f2f1ff14298,delivered,2023-04-07 15:48:17,2023-04-07 16:08:45,2023-04-13 20:06:37,2023-04-25 00:00:00,1,...,credit_card,6.0,88.78,books_general_interest,611.0,22.0,22.0,23.0,books_general_interest,1
118430,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2023-04-04 08:20:22,2023-04-04 08:35:12,2023-04-11 18:54:45,2023-04-20 00:00:00,1,...,credit_card,3.0,129.06,sports_leisure,1211.0,25.0,24.0,22.0,sports_leisure,1
118431,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fed4434add09a6f332ea398efd656a5c,delivered,2023-04-08 20:11:50,2023-04-08 20:30:03,2023-05-09 19:03:15,2023-05-02 00:00:00,1,...,credit_card,5.0,56.04,health_beauty,870.0,25.0,20.0,18.0,health_beauty,1
118432,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,e31ec91cea1ecf97797787471f98a8c2,delivered,2022-11-03 21:08:33,2022-11-03 21:31:20,2022-11-16 19:58:39,2022-12-05 00:00:00,1,...,credit_card,2.0,711.07,watches_gifts,710.0,19.0,13.0,14.0,watches_gifts,1


## 결측치 처리

In [317]:
# 데이터프레임 df의 컬럼별 결측치의 수를 확인하세요.
df.isnull().sum()

customer_id                         0
customer_unique_id                  0
customer_zip_code_prefix            0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 176
order_delivered_customer_date    3397
order_estimated_delivery_date       0
total_orders                        0
product_id                        830
seller_id                         830
price                             830
freight_value                     830
total_order_items                   0
payment_sequential                  3
payment_type                        3
payment_installments                3
payment_value                       3
product_category_name            2528
product_weight_g                  850
product_length_cm                 850
product_height_cm                 850
product_width_cm                  850
product_category_name_english    2553
is_once                             0
dtype: int64

In [318]:
# product_id~freight_value가 nan값인 행을 확인하니, order_status가 모두 unavilable, canceled임을 알 수 있다
df[df['product_id'].isna()]

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,payment_type,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,is_once
52,f34a6e874087ec1f0e3dab9fdf659c5d,233896de79986082f1f479f1f85281cb,38300,6e98de3a85c84ead6689189b825d35b5,canceled,2023-03-15 10:07:02,2023-03-15 10:29:33,NaN,2023-04-09 00:00:00,1,...,credit_card,1.0,73.16,NaN,NaN,NaN,NaN,NaN,NaN,1
341,5bfe800011656c0afb81db64519982db,0071f46a072a9ae25bbe4438b15efe9c,16011,df8c077268f7f3baaac0892eb3143642,unavailable,2022-02-01 00:04:17,2022-02-01 00:30:55,NaN,2022-03-10 00:00:00,1,...,credit_card,4.0,174.54,NaN,NaN,NaN,NaN,NaN,NaN,1
438,b08064e24083fee8fbe8797902b07ecd,035f60af6e7d7f78470e9443be08d339,28495,c609f82bcf7a90292a5940205ebd7e93,unavailable,2023-05-13 16:45:55,2023-05-13 17:11:58,NaN,2023-06-05 00:00:00,1,...,credit_card,2.0,117.78,NaN,NaN,NaN,NaN,NaN,NaN,1
448,536f46cc0f2f2b1e40d056f7998f0254,340152332a04149987a705602615f0d0,87030,cb4a79c1e6c9ae44302861e7602cc449,unavailable,2022-12-06 15:46:07,2022-12-06 15:57:29,NaN,2023-01-03 00:00:00,1,...,credit_card,1.0,66.02,NaN,NaN,NaN,NaN,NaN,NaN,1
656,8118922685d2e2c0205f060be4f2579c,d0e87d00021530383c16452a39a393ba,45810,af264f3527e94e431f0dcd56cd6b406d,unavailable,2022-07-12 14:49:11,2022-07-12 15:04:10,NaN,2022-08-11 00:00:00,1,...,credit_card,1.0,46.69,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117027,7353b0fb8e8d9675e3a704c60ca44ebe,21c933c8dd97d088e64c50988c90ccf5,5017,2f634e2cebf8c0283e7ef0989f77d217,unavailable,2022-09-27 20:55:33,2022-09-28 01:32:50,NaN,2022-10-27 00:00:00,1,...,credit_card,12.0,615.53,NaN,NaN,NaN,NaN,NaN,NaN,1
117087,5effe9491d0f5a1c12ee4a384ec09bdf,e9429d622d9e764ca60ba01bc252b0a8,8690,2b0edc4c59d83dcef85466718c36a317,unavailable,2022-09-29 19:17:10,2022-09-29 20:00:07,NaN,2022-10-17 00:00:00,1,...,credit_card,1.0,112.91,NaN,NaN,NaN,NaN,NaN,NaN,1
117377,75c6e03e7206bd1dd59f427c9f75142a,ff36be26206fffe1eb37afd54c70e18b,13057,81b7c7bbc8ec003eeb67d87441a6a148,canceled,2023-08-20 09:03:23,NaN,NaN,2023-09-03 00:00:00,3,...,voucher,1.0,92.76,NaN,NaN,NaN,NaN,NaN,NaN,0
117482,1761db2f0d235bbfa1c281093198af0c,21c2c79d24e2d8e3496aabb1f65d85c6,20270,8b08f0e729f58529ed03e763270f78d5,unavailable,2022-08-24 21:02:47,2022-08-24 21:15:36,NaN,2022-09-27 00:00:00,1,...,credit_card,1.0,136.50,NaN,NaN,NaN,NaN,NaN,NaN,1


In [319]:
# order_status가 unavailable, canceled인 것은 필요없으므로 제거해준다.
df.dropna(subset=['product_id'],inplace=True)
df

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,payment_type,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,is_once
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,00e7ee1b050b8499577073aeb2a297a1,delivered,2022-05-16 15:05:35,2022-05-16 15:22:12,2022-05-25 10:35:35,2022-06-05 00:00:00,1,...,credit_card,2.0,146.87,office_furniture,8683.0,54.0,64.0,31.0,office_furniture,1
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,29150127e6685892b6eab3eec79f59c7,delivered,2023-01-12 20:48:24,2023-01-12 20:58:32,2023-01-29 12:41:19,2023-02-06 00:00:00,1,...,credit_card,8.0,335.48,housewares,10150.0,89.0,15.0,40.0,housewares,1
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2023-05-19 16:07:45,2023-05-20 16:19:10,2023-06-14 17:58:51,2023-06-13 00:00:00,1,...,credit_card,7.0,157.73,office_furniture,8267.0,52.0,52.0,17.0,office_furniture,1
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,951670f92359f4fe4a63112aa7306eba,delivered,2023-03-13 16:06:38,2023-03-13 17:29:19,2023-03-28 16:04:25,2023-04-10 00:00:00,1,...,credit_card,1.0,173.30,office_furniture,12160.0,56.0,51.0,28.0,office_furniture,1
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2023-07-29 09:51:30,2023-07-29 10:10:09,2023-08-09 20:55:48,2023-08-15 00:00:00,1,...,credit_card,8.0,252.25,home_confort,5200.0,45.0,15.0,35.0,home_confort,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,6760e20addcf0121e9d58f2f1ff14298,delivered,2023-04-07 15:48:17,2023-04-07 16:08:45,2023-04-13 20:06:37,2023-04-25 00:00:00,1,...,credit_card,6.0,88.78,books_general_interest,611.0,22.0,22.0,23.0,books_general_interest,1
118430,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2023-04-04 08:20:22,2023-04-04 08:35:12,2023-04-11 18:54:45,2023-04-20 00:00:00,1,...,credit_card,3.0,129.06,sports_leisure,1211.0,25.0,24.0,22.0,sports_leisure,1
118431,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fed4434add09a6f332ea398efd656a5c,delivered,2023-04-08 20:11:50,2023-04-08 20:30:03,2023-05-09 19:03:15,2023-05-02 00:00:00,1,...,credit_card,5.0,56.04,health_beauty,870.0,25.0,20.0,18.0,health_beauty,1
118432,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,e31ec91cea1ecf97797787471f98a8c2,delivered,2022-11-03 21:08:33,2022-11-03 21:31:20,2022-11-16 19:58:39,2022-12-05 00:00:00,1,...,credit_card,2.0,711.07,watches_gifts,710.0,19.0,13.0,14.0,watches_gifts,1


In [320]:
# 행 삭제가 확인됨
df.isnull().sum()

customer_id                         0
customer_unique_id                  0
customer_zip_code_prefix            0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  15
order_delivered_customer_date    2567
order_estimated_delivery_date       0
total_orders                        0
product_id                          0
seller_id                           0
price                               0
freight_value                       0
total_order_items                   0
payment_sequential                  3
payment_type                        3
payment_installments                3
payment_value                       3
product_category_name            1698
product_weight_g                   20
product_length_cm                  20
product_height_cm                  20
product_width_cm                   20
product_category_name_english    1723
is_once                             0
dtype: int64

In [321]:
# 'product_category_name' 열에서 NaN 값을 'unknown'으로 대체
df['product_category_name'].fillna('unknown', inplace=True)

# 'product_category_name_english' 열에서도 NaN 값을 'unknown'으로 대체
df['product_category_name_english'].fillna('unknown', inplace=True)

In [322]:
# 대체된 것을 확인
df.isnull().sum()

customer_id                         0
customer_unique_id                  0
customer_zip_code_prefix            0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  15
order_delivered_customer_date    2567
order_estimated_delivery_date       0
total_orders                        0
product_id                          0
seller_id                           0
price                               0
freight_value                       0
total_order_items                   0
payment_sequential                  3
payment_type                        3
payment_installments                3
payment_value                       3
product_category_name               0
product_weight_g                   20
product_length_cm                  20
product_height_cm                  20
product_width_cm                   20
product_category_name_english       0
is_once                             0
dtype: int64

In [323]:
# product_length_cm~product_width_cm와 같은 제품에 대한 정보는 0으로 처리
df['product_weight_g'].fillna(0, inplace=True)
df['product_length_cm'].fillna(0, inplace=True)
df['product_height_cm'].fillna(0, inplace=True)
df['product_width_cm'].fillna(0, inplace=True)

In [324]:
# 대체된 것을 확인
df.isnull().sum()

customer_id                         0
customer_unique_id                  0
customer_zip_code_prefix            0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  15
order_delivered_customer_date    2567
order_estimated_delivery_date       0
total_orders                        0
product_id                          0
seller_id                           0
price                               0
freight_value                       0
total_order_items                   0
payment_sequential                  3
payment_type                        3
payment_installments                3
payment_value                       3
product_category_name               0
product_weight_g                    0
product_length_cm                   0
product_height_cm                   0
product_width_cm                    0
product_category_name_english       0
is_once                             0
dtype: int64

In [325]:
# payment_sequential~payment_value가 nan값인 행이 3개만 존재
df[df['payment_value'].isna()]

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,payment_type,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,is_once
25207,86dc2ffce2dfff336de2f386a786e574,830d5b7aaa3b6f1e9ad63703bec97d23,14600,bfbd0f9bdef84302105ad712db648a6c,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-09 07:47:38,2016-10-04 00:00:00,1,...,NaN,NaN,NaN,health_beauty,1000.0,16.0,16.0,16.0,health_beauty,1
25208,86dc2ffce2dfff336de2f386a786e574,830d5b7aaa3b6f1e9ad63703bec97d23,14600,bfbd0f9bdef84302105ad712db648a6c,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-09 07:47:38,2016-10-04 00:00:00,1,...,NaN,NaN,NaN,health_beauty,1000.0,16.0,16.0,16.0,health_beauty,1
25209,86dc2ffce2dfff336de2f386a786e574,830d5b7aaa3b6f1e9ad63703bec97d23,14600,bfbd0f9bdef84302105ad712db648a6c,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-09 07:47:38,2016-10-04 00:00:00,1,...,NaN,NaN,NaN,health_beauty,1000.0,16.0,16.0,16.0,health_beauty,1


In [326]:
# 3개밖에 없고 1회 방문자이므로 행 제거
df.dropna(subset=['payment_value'],inplace=True)

In [327]:
# 제거된 것을 확인
df.isnull().sum()

customer_id                         0
customer_unique_id                  0
customer_zip_code_prefix            0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  15
order_delivered_customer_date    2567
order_estimated_delivery_date       0
total_orders                        0
product_id                          0
seller_id                           0
price                               0
freight_value                       0
total_order_items                   0
payment_sequential                  0
payment_type                        0
payment_installments                0
payment_value                       0
product_category_name               0
product_weight_g                    0
product_length_cm                   0
product_height_cm                   0
product_width_cm                    0
product_category_name_english       0
is_once                             0
dtype: int64

### 날짜 관련 열 전처리

In [328]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])

In [329]:
# order_approved_at열 중 nan값인 행들 확인 -> 모두 배송은 완료되긴 함
df[df['order_approved_at'].isna()]

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,payment_type,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,is_once
10720,0bf35cac6cc7327065da879e2d90fae8,c4c0011e639bdbcf26059ddc38bd3c18,13225,d77031d6a3c8a52f019764e68f211c69,delivered,2022-02-18 11:04:19,NaT,2022-03-02 16:15:23,2022-03-22,1,...,boleto,1.0,39.95,sports_leisure,300.0,16.0,15.0,15.0,sports_leisure,1
16565,1e101e0daffaddce8159d25a8e53f2b2,c8822fce1d0bfa7ddf0da24fff947172,27945,12a95a3c06dbaec84bcfb0e2da5d228a,delivered,2022-02-17 13:05:55,NaT,2022-03-02 11:09:19,2022-03-20,1,...,boleto,1.0,95.76,cool_stuff,1200.0,42.0,25.0,15.0,cool_stuff,1
24896,d5de688c321096d15508faae67a27051,d49f3dae6bad25d05160fc17aca5942d,36400,7002a78c79c519ac54022d4f8a65e6e8,delivered,2022-01-19 22:26:59,NaT,2022-02-06 14:22:19,2022-03-16,1,...,boleto,1.0,60.42,furniture_decor,600.0,30.0,3.0,20.0,furniture_decor,1
27085,07a2a7e0f63fd8cb757ed77d4245623c,79af1bbf230a2630487975aa5d7d6220,37660,51eb2eebd5d76a24625b31c33dd41449,delivered,2022-02-18 15:52:27,NaT,2022-03-07 13:57:47,2022-03-29,1,...,boleto,1.0,77.06,furniture_decor,1200.0,30.0,5.0,20.0,furniture_decor,1
32720,68d081753ad4fe22fc4d410a9eb1ca01,2e0a2166aa23da2472c6a60c4af6f7a6,3573,d69e5d356402adc8cf17e08b5033acfb,delivered,2022-02-19 01:28:47,NaT,2022-03-02 03:41:58,2022-03-27,2,...,boleto,1.0,163.43,furniture_decor,1700.0,33.0,8.0,23.0,furniture_decor,0
46838,d85919cb3c0529589c6fa617f5f43281,c094ac95fcd52f821809ec232a7a6956,95795,3c0b8706b065f9919d0505d3b3343881,delivered,2022-02-17 15:53:27,NaT,2022-03-03 11:47:47,2022-03-23,1,...,boleto,1.0,157.19,bed_bath_table,2700.0,42.0,11.0,36.0,bed_bath_table,1
49485,74bebaf46603f9340e3b50c6b086f992,f79be7c08dd24b72d34634f1b89333a4,65110,2babbb4b15e6d2dfe95e2de765c97bce,delivered,2022-02-18 17:15:03,NaT,2022-03-03 18:43:43,2022-03-31,1,...,boleto,1.0,106.81,cool_stuff,1200.0,42.0,25.0,15.0,cool_stuff,1
52480,684cb238dc5b5d6366244e0e0776b450,6ff8b0d7b35d5c945633b8d60165691b,11030,c1d4211b3dae76144deccd6c74144a88,delivered,2022-01-19 12:48:08,NaT,2022-01-30 18:16:01,2022-03-01,1,...,boleto,1.0,54.51,sports_leisure,700.0,26.0,16.0,21.0,sports_leisure,1
62999,a3d3c38e58b9d2dfb9207cab690b6310,5a4fa4919cbf2b049e72be460a380e5b,35620,2eecb0d85f281280f79fa00f9cec1a95,delivered,2022-02-17 17:21:55,NaT,2022-03-03 12:16:03,2022-03-20,1,...,boleto,1.0,154.23,garden_tools,4300.0,36.0,13.0,20.0,garden_tools,1
71524,2127dc6603ac33544953ef05ec155771,8a9a08c7ca8900a200d83cf838a07e0b,6708,e04abd8149ef81b95221e88f6ed9ab6a,delivered,2022-02-18 14:40:00,NaT,2022-03-01 13:25:33,2022-03-17,1,...,boleto,1.0,349.01,small_appliances,20800.0,75.0,40.0,40.0,small_appliances,1


In [330]:
# order_purchase_timestamp열과 order_approved_at열 사이 평균적인 기간을 고려해서 order_approved_at열의 누락된 것을 채우기
# 1단계: 차이 계산
time_differences = df['order_approved_at'] - df['order_purchase_timestamp']

# 2단계: 평균 시간 차이 계산 (초 단위로 반올림)
average_difference_seconds = round(time_differences.dt.total_seconds().mean())

# '초' 단위의 평균 차이를 다시 timedelta로 변환
average_difference = pd.to_timedelta(average_difference_seconds, unit='s')

# 3단계: 누락된 'order_approved_at' 값을 채움
df.loc[df['order_approved_at'].isna(), 'order_approved_at'] = df['order_purchase_timestamp'] + average_difference

In [331]:
average_difference

Timedelta('0 days 10:29:43')

In [332]:
# 잘 대체됐는지 예시를 통해 확인
df[df['customer_unique_id']=='c8822fce1d0bfa7ddf0da24fff947172']

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,payment_type,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,is_once
16565,1e101e0daffaddce8159d25a8e53f2b2,c8822fce1d0bfa7ddf0da24fff947172,27945,12a95a3c06dbaec84bcfb0e2da5d228a,delivered,2022-02-17 13:05:55,2022-02-17 23:35:38,2022-03-02 11:09:19,2022-03-20,1,...,boleto,1.0,95.76,cool_stuff,1200.0,42.0,25.0,15.0,cool_stuff,1


In [333]:
# 대체된 것을 확인
df.isnull().sum()

customer_id                         0
customer_unique_id                  0
customer_zip_code_prefix            0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_approved_at                   0
order_delivered_customer_date    2567
order_estimated_delivery_date       0
total_orders                        0
product_id                          0
seller_id                           0
price                               0
freight_value                       0
total_order_items                   0
payment_sequential                  0
payment_type                        0
payment_installments                0
payment_value                       0
product_category_name               0
product_weight_g                    0
product_length_cm                   0
product_height_cm                   0
product_width_cm                    0
product_category_name_english       0
is_once                             0
dtype: int64

In [334]:
# order_delivered_customer_date 중 nan값인 행들 확인 -> 모두 배송은 완료되긴 함
df[df['order_delivered_customer_date'].isna()]

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,payment_type,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,is_once
53,c132855c926907970dcf6f2bf0b33a24,a8ae36a2bb6c2bbc3b5d62ede131c9ef,18740,b5172d4a722c0ebb34c29bbbb7a3cb42,invoiced,2023-04-17 13:14:13,2023-04-18 05:31:51,NaT,2023-05-24,1,...,boleto,1.0,143.25,consoles_games,300.0,25.0,11.0,18.0,consoles_games,1
84,4c7241af24b5344cb01fe687643de4fe,b157c176c3fe04914fde33f2dc8b878a,60336,ad380680e87dea0f2abf5cd5bace626c,shipped,2023-01-21 15:54:02,2023-01-22 13:52:41,NaT,2023-02-20,1,...,credit_card,1.0,74.38,furniture_decor,300.0,69.0,11.0,11.0,furniture_decor,1
95,f6529ffebe6b3440d45d89604a4239ac,e5dbefdfdf3eff75c8e6cd655f128279,26272,cc07a8fdd3a8e94d683c8142a117dbc1,shipped,2023-01-11 17:11:11,2023-01-12 17:49:45,NaT,2023-02-09,1,...,voucher,1.0,137.60,unknown,400.0,20.0,12.0,15.0,unknown,1
119,ae76a4650235ab18764708174f1da31e,2b6082a140c439e2df870c85b0aa5e88,2983,4c33ec562d9dc5655e160913aa86eb53,shipped,2022-08-11 23:47:48,2022-08-12 00:03:51,NaT,2022-09-11,1,...,credit_card,5.0,58.59,furniture_decor,600.0,30.0,3.0,20.0,furniture_decor,1
134,8891eb5ca0e28df961b2b5b8f3c0eb23,13ac6da47fef1b17b16f511e03b57918,2632,ff536d93ae4214b4d51c2894ccfc569f,shipped,2022-12-21 09:01:00,2022-12-22 12:33:26,NaT,2023-01-10,2,...,credit_card,1.0,47.27,sports_leisure,433.0,16.0,16.0,16.0,sports_leisure,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118203,6a65b5d6d4f4c9dfda93798cf014ee47,5b66137675c164a7df3b639bf1e86134,3959,8c0dd54ff6a273a02de12494c6bf299f,shipped,2023-07-17 22:35:46,2023-07-18 21:22:25,NaT,2023-08-02,1,...,credit_card,1.0,20.86,telephony,125.0,19.0,5.0,11.0,telephony,1
118238,9ce70b8dafe76524f3d5e5e68cbf29f5,6edd9e4a22b7ab02db2a87f13bfd2d2a,14093,e5e59fe47003046e2ed48ebb641564b2,shipped,2023-04-27 13:52:12,2023-04-27 14:09:00,NaT,2023-05-23,1,...,credit_card,2.0,98.70,furniture_decor,12300.0,40.0,40.0,40.0,furniture_decor,1
118246,2a6fc1968c4a29c9c258bb1994afbc7f,205a1c797b2c6e626d560900a3ebde6b,27210,12d9fe1a3efa51a7fae337cba819adc1,shipped,2022-02-08 13:07:19,2022-02-09 02:50:42,NaT,2022-03-17,1,...,boleto,1.0,39.42,housewares,300.0,25.0,2.0,17.0,housewares,1
118317,f83083fcbc51d12f8279b5713c2d4b4d,0465f164e35aba70f70e911b252513a9,77807,67384817c871ae183b24f9a0a6eb9bbc,shipped,2023-01-25 12:12:37,2023-01-25 12:31:27,NaT,2023-03-01,1,...,credit_card,8.0,554.96,watches_gifts,700.0,20.0,13.0,13.0,watches_gifts,1


## 필요한 열 생성

### 배송지연 여부 열 생성 (예상 배송일, 실제 배송일)

In [335]:
df['is_delayed'] = (df['order_delivered_customer_date'] > df['order_estimated_delivery_date']).apply(lambda x: 1 if x else 0)
df  

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,is_once,is_delayed
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,00e7ee1b050b8499577073aeb2a297a1,delivered,2022-05-16 15:05:35,2022-05-16 15:22:12,2022-05-25 10:35:35,2022-06-05,1,...,2.0,146.87,office_furniture,8683.0,54.0,64.0,31.0,office_furniture,1,0
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,29150127e6685892b6eab3eec79f59c7,delivered,2023-01-12 20:48:24,2023-01-12 20:58:32,2023-01-29 12:41:19,2023-02-06,1,...,8.0,335.48,housewares,10150.0,89.0,15.0,40.0,housewares,1,0
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2023-05-19 16:07:45,2023-05-20 16:19:10,2023-06-14 17:58:51,2023-06-13,1,...,7.0,157.73,office_furniture,8267.0,52.0,52.0,17.0,office_furniture,1,1
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,951670f92359f4fe4a63112aa7306eba,delivered,2023-03-13 16:06:38,2023-03-13 17:29:19,2023-03-28 16:04:25,2023-04-10,1,...,1.0,173.30,office_furniture,12160.0,56.0,51.0,28.0,office_furniture,1,0
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2023-07-29 09:51:30,2023-07-29 10:10:09,2023-08-09 20:55:48,2023-08-15,1,...,8.0,252.25,home_confort,5200.0,45.0,15.0,35.0,home_confort,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,6760e20addcf0121e9d58f2f1ff14298,delivered,2023-04-07 15:48:17,2023-04-07 16:08:45,2023-04-13 20:06:37,2023-04-25,1,...,6.0,88.78,books_general_interest,611.0,22.0,22.0,23.0,books_general_interest,1,0
118430,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2023-04-04 08:20:22,2023-04-04 08:35:12,2023-04-11 18:54:45,2023-04-20,1,...,3.0,129.06,sports_leisure,1211.0,25.0,24.0,22.0,sports_leisure,1,0
118431,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fed4434add09a6f332ea398efd656a5c,delivered,2023-04-08 20:11:50,2023-04-08 20:30:03,2023-05-09 19:03:15,2023-05-02,1,...,5.0,56.04,health_beauty,870.0,25.0,20.0,18.0,health_beauty,1,1
118432,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,e31ec91cea1ecf97797787471f98a8c2,delivered,2022-11-03 21:08:33,2022-11-03 21:31:20,2022-11-16 19:58:39,2022-12-05,1,...,2.0,711.07,watches_gifts,710.0,19.0,13.0,14.0,watches_gifts,1,0


### 구매한 시점과 예정 배송일의 차이

In [336]:
time_differences= df['order_estimated_delivery_date'] - df['order_purchase_timestamp'] 
df['duration_purchase_estimated'] = round(time_differences.dt.total_seconds()) 
df  

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,is_once,is_delayed,duration_purchase_estimated
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,00e7ee1b050b8499577073aeb2a297a1,delivered,2022-05-16 15:05:35,2022-05-16 15:22:12,2022-05-25 10:35:35,2022-06-05,1,...,146.87,office_furniture,8683.0,54.0,64.0,31.0,office_furniture,1,0,1673665.0
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,29150127e6685892b6eab3eec79f59c7,delivered,2023-01-12 20:48:24,2023-01-12 20:58:32,2023-01-29 12:41:19,2023-02-06,1,...,335.48,housewares,10150.0,89.0,15.0,40.0,housewares,1,0,2085096.0
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2023-05-19 16:07:45,2023-05-20 16:19:10,2023-06-14 17:58:51,2023-06-13,1,...,157.73,office_furniture,8267.0,52.0,52.0,17.0,office_furniture,1,1,2101935.0
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,951670f92359f4fe4a63112aa7306eba,delivered,2023-03-13 16:06:38,2023-03-13 17:29:19,2023-03-28 16:04:25,2023-04-10,1,...,173.30,office_furniture,12160.0,56.0,51.0,28.0,office_furniture,1,0,2361202.0
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2023-07-29 09:51:30,2023-07-29 10:10:09,2023-08-09 20:55:48,2023-08-15,1,...,252.25,home_confort,5200.0,45.0,15.0,35.0,home_confort,1,0,1433310.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,6760e20addcf0121e9d58f2f1ff14298,delivered,2023-04-07 15:48:17,2023-04-07 16:08:45,2023-04-13 20:06:37,2023-04-25,1,...,88.78,books_general_interest,611.0,22.0,22.0,23.0,books_general_interest,1,0,1498303.0
118430,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2023-04-04 08:20:22,2023-04-04 08:35:12,2023-04-11 18:54:45,2023-04-20,1,...,129.06,sports_leisure,1211.0,25.0,24.0,22.0,sports_leisure,1,0,1352378.0
118431,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fed4434add09a6f332ea398efd656a5c,delivered,2023-04-08 20:11:50,2023-04-08 20:30:03,2023-05-09 19:03:15,2023-05-02,1,...,56.04,health_beauty,870.0,25.0,20.0,18.0,health_beauty,1,1,2000890.0
118432,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,e31ec91cea1ecf97797787471f98a8c2,delivered,2022-11-03 21:08:33,2022-11-03 21:31:20,2022-11-16 19:58:39,2022-12-05,1,...,711.07,watches_gifts,710.0,19.0,13.0,14.0,watches_gifts,1,0,2688687.0


### 구매한 시점과 승인날 차이

In [337]:
time_differences= df['order_approved_at'] - df['order_purchase_timestamp'] 
df['duration_purchase_approved'] = round(time_differences.dt.total_seconds()) 
df  

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,is_once,is_delayed,duration_purchase_estimated,duration_purchase_approved
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,00e7ee1b050b8499577073aeb2a297a1,delivered,2022-05-16 15:05:35,2022-05-16 15:22:12,2022-05-25 10:35:35,2022-06-05,1,...,office_furniture,8683.0,54.0,64.0,31.0,office_furniture,1,0,1673665.0,997.0
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,29150127e6685892b6eab3eec79f59c7,delivered,2023-01-12 20:48:24,2023-01-12 20:58:32,2023-01-29 12:41:19,2023-02-06,1,...,housewares,10150.0,89.0,15.0,40.0,housewares,1,0,2085096.0,608.0
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2023-05-19 16:07:45,2023-05-20 16:19:10,2023-06-14 17:58:51,2023-06-13,1,...,office_furniture,8267.0,52.0,52.0,17.0,office_furniture,1,1,2101935.0,87085.0
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,951670f92359f4fe4a63112aa7306eba,delivered,2023-03-13 16:06:38,2023-03-13 17:29:19,2023-03-28 16:04:25,2023-04-10,1,...,office_furniture,12160.0,56.0,51.0,28.0,office_furniture,1,0,2361202.0,4961.0
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2023-07-29 09:51:30,2023-07-29 10:10:09,2023-08-09 20:55:48,2023-08-15,1,...,home_confort,5200.0,45.0,15.0,35.0,home_confort,1,0,1433310.0,1119.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,6760e20addcf0121e9d58f2f1ff14298,delivered,2023-04-07 15:48:17,2023-04-07 16:08:45,2023-04-13 20:06:37,2023-04-25,1,...,books_general_interest,611.0,22.0,22.0,23.0,books_general_interest,1,0,1498303.0,1228.0
118430,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2023-04-04 08:20:22,2023-04-04 08:35:12,2023-04-11 18:54:45,2023-04-20,1,...,sports_leisure,1211.0,25.0,24.0,22.0,sports_leisure,1,0,1352378.0,890.0
118431,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fed4434add09a6f332ea398efd656a5c,delivered,2023-04-08 20:11:50,2023-04-08 20:30:03,2023-05-09 19:03:15,2023-05-02,1,...,health_beauty,870.0,25.0,20.0,18.0,health_beauty,1,1,2000890.0,1093.0
118432,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,e31ec91cea1ecf97797787471f98a8c2,delivered,2022-11-03 21:08:33,2022-11-03 21:31:20,2022-11-16 19:58:39,2022-12-05,1,...,watches_gifts,710.0,19.0,13.0,14.0,watches_gifts,1,0,2688687.0,1367.0


### 구매한 시점과 실제 배송일 차이

In [338]:
time_differences= df['order_delivered_customer_date'] - df['order_purchase_timestamp'] 
df['duration_purchase_delivered'] = round(time_differences.dt.total_seconds()) 
df  

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,is_once,is_delayed,duration_purchase_estimated,duration_purchase_approved,duration_purchase_delivered
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,00e7ee1b050b8499577073aeb2a297a1,delivered,2022-05-16 15:05:35,2022-05-16 15:22:12,2022-05-25 10:35:35,2022-06-05,1,...,8683.0,54.0,64.0,31.0,office_furniture,1,0,1673665.0,997.0,761400.0
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,29150127e6685892b6eab3eec79f59c7,delivered,2023-01-12 20:48:24,2023-01-12 20:58:32,2023-01-29 12:41:19,2023-02-06,1,...,10150.0,89.0,15.0,40.0,housewares,1,0,2085096.0,608.0,1439575.0
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2023-05-19 16:07:45,2023-05-20 16:19:10,2023-06-14 17:58:51,2023-06-13,1,...,8267.0,52.0,52.0,17.0,office_furniture,1,1,2101935.0,87085.0,2253066.0
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,951670f92359f4fe4a63112aa7306eba,delivered,2023-03-13 16:06:38,2023-03-13 17:29:19,2023-03-28 16:04:25,2023-04-10,1,...,12160.0,56.0,51.0,28.0,office_furniture,1,0,2361202.0,4961.0,1295867.0
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2023-07-29 09:51:30,2023-07-29 10:10:09,2023-08-09 20:55:48,2023-08-15,1,...,5200.0,45.0,15.0,35.0,home_confort,1,0,1433310.0,1119.0,990258.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,6760e20addcf0121e9d58f2f1ff14298,delivered,2023-04-07 15:48:17,2023-04-07 16:08:45,2023-04-13 20:06:37,2023-04-25,1,...,611.0,22.0,22.0,23.0,books_general_interest,1,0,1498303.0,1228.0,533900.0
118430,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2023-04-04 08:20:22,2023-04-04 08:35:12,2023-04-11 18:54:45,2023-04-20,1,...,1211.0,25.0,24.0,22.0,sports_leisure,1,0,1352378.0,890.0,642863.0
118431,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fed4434add09a6f332ea398efd656a5c,delivered,2023-04-08 20:11:50,2023-04-08 20:30:03,2023-05-09 19:03:15,2023-05-02,1,...,870.0,25.0,20.0,18.0,health_beauty,1,1,2000890.0,1093.0,2674285.0
118432,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,e31ec91cea1ecf97797787471f98a8c2,delivered,2022-11-03 21:08:33,2022-11-03 21:31:20,2022-11-16 19:58:39,2022-12-05,1,...,710.0,19.0,13.0,14.0,watches_gifts,1,0,2688687.0,1367.0,1119006.0


### 승인날과 실제 배송일 차이

In [339]:
time_differences= df['order_delivered_customer_date'] - df['order_approved_at'] 
df['duration_approved_delivered'] = round(time_differences.dt.total_seconds()) 
df  

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,is_once,is_delayed,duration_purchase_estimated,duration_purchase_approved,duration_purchase_delivered,duration_approved_delivered
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,00e7ee1b050b8499577073aeb2a297a1,delivered,2022-05-16 15:05:35,2022-05-16 15:22:12,2022-05-25 10:35:35,2022-06-05,1,...,54.0,64.0,31.0,office_furniture,1,0,1673665.0,997.0,761400.0,760403.0
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,29150127e6685892b6eab3eec79f59c7,delivered,2023-01-12 20:48:24,2023-01-12 20:58:32,2023-01-29 12:41:19,2023-02-06,1,...,89.0,15.0,40.0,housewares,1,0,2085096.0,608.0,1439575.0,1438967.0
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2023-05-19 16:07:45,2023-05-20 16:19:10,2023-06-14 17:58:51,2023-06-13,1,...,52.0,52.0,17.0,office_furniture,1,1,2101935.0,87085.0,2253066.0,2165981.0
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,951670f92359f4fe4a63112aa7306eba,delivered,2023-03-13 16:06:38,2023-03-13 17:29:19,2023-03-28 16:04:25,2023-04-10,1,...,56.0,51.0,28.0,office_furniture,1,0,2361202.0,4961.0,1295867.0,1290906.0
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2023-07-29 09:51:30,2023-07-29 10:10:09,2023-08-09 20:55:48,2023-08-15,1,...,45.0,15.0,35.0,home_confort,1,0,1433310.0,1119.0,990258.0,989139.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,6760e20addcf0121e9d58f2f1ff14298,delivered,2023-04-07 15:48:17,2023-04-07 16:08:45,2023-04-13 20:06:37,2023-04-25,1,...,22.0,22.0,23.0,books_general_interest,1,0,1498303.0,1228.0,533900.0,532672.0
118430,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2023-04-04 08:20:22,2023-04-04 08:35:12,2023-04-11 18:54:45,2023-04-20,1,...,25.0,24.0,22.0,sports_leisure,1,0,1352378.0,890.0,642863.0,641973.0
118431,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fed4434add09a6f332ea398efd656a5c,delivered,2023-04-08 20:11:50,2023-04-08 20:30:03,2023-05-09 19:03:15,2023-05-02,1,...,25.0,20.0,18.0,health_beauty,1,1,2000890.0,1093.0,2674285.0,2673192.0
118432,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,e31ec91cea1ecf97797787471f98a8c2,delivered,2022-11-03 21:08:33,2022-11-03 21:31:20,2022-11-16 19:58:39,2022-12-05,1,...,19.0,13.0,14.0,watches_gifts,1,0,2688687.0,1367.0,1119006.0,1117639.0


### 승인날과 예상 배송일 차이

In [340]:
time_differences= df['order_estimated_delivery_date'] - df['order_approved_at'] 
df['duration_approved_estimated'] = round(time_differences.dt.total_seconds()) 
df  

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,product_height_cm,product_width_cm,product_category_name_english,is_once,is_delayed,duration_purchase_estimated,duration_purchase_approved,duration_purchase_delivered,duration_approved_delivered,duration_approved_estimated
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,00e7ee1b050b8499577073aeb2a297a1,delivered,2022-05-16 15:05:35,2022-05-16 15:22:12,2022-05-25 10:35:35,2022-06-05,1,...,64.0,31.0,office_furniture,1,0,1673665.0,997.0,761400.0,760403.0,1672668.0
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,29150127e6685892b6eab3eec79f59c7,delivered,2023-01-12 20:48:24,2023-01-12 20:58:32,2023-01-29 12:41:19,2023-02-06,1,...,15.0,40.0,housewares,1,0,2085096.0,608.0,1439575.0,1438967.0,2084488.0
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2023-05-19 16:07:45,2023-05-20 16:19:10,2023-06-14 17:58:51,2023-06-13,1,...,52.0,17.0,office_furniture,1,1,2101935.0,87085.0,2253066.0,2165981.0,2014850.0
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,951670f92359f4fe4a63112aa7306eba,delivered,2023-03-13 16:06:38,2023-03-13 17:29:19,2023-03-28 16:04:25,2023-04-10,1,...,51.0,28.0,office_furniture,1,0,2361202.0,4961.0,1295867.0,1290906.0,2356241.0
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2023-07-29 09:51:30,2023-07-29 10:10:09,2023-08-09 20:55:48,2023-08-15,1,...,15.0,35.0,home_confort,1,0,1433310.0,1119.0,990258.0,989139.0,1432191.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,6760e20addcf0121e9d58f2f1ff14298,delivered,2023-04-07 15:48:17,2023-04-07 16:08:45,2023-04-13 20:06:37,2023-04-25,1,...,22.0,23.0,books_general_interest,1,0,1498303.0,1228.0,533900.0,532672.0,1497075.0
118430,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2023-04-04 08:20:22,2023-04-04 08:35:12,2023-04-11 18:54:45,2023-04-20,1,...,24.0,22.0,sports_leisure,1,0,1352378.0,890.0,642863.0,641973.0,1351488.0
118431,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fed4434add09a6f332ea398efd656a5c,delivered,2023-04-08 20:11:50,2023-04-08 20:30:03,2023-05-09 19:03:15,2023-05-02,1,...,20.0,18.0,health_beauty,1,1,2000890.0,1093.0,2674285.0,2673192.0,1999797.0
118432,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,e31ec91cea1ecf97797787471f98a8c2,delivered,2022-11-03 21:08:33,2022-11-03 21:31:20,2022-11-16 19:58:39,2022-12-05,1,...,13.0,14.0,watches_gifts,1,0,2688687.0,1367.0,1119006.0,1117639.0,2687320.0


## 불필요한 열 drop

In [341]:
df.isnull().sum()

customer_id                         0
customer_unique_id                  0
customer_zip_code_prefix            0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_approved_at                   0
order_delivered_customer_date    2567
order_estimated_delivery_date       0
total_orders                        0
product_id                          0
seller_id                           0
price                               0
freight_value                       0
total_order_items                   0
payment_sequential                  0
payment_type                        0
payment_installments                0
payment_value                       0
product_category_name               0
product_weight_g                    0
product_length_cm                   0
product_height_cm                   0
product_width_cm                    0
product_category_name_english       0
is_once                             0
is_delayed  

In [342]:
df.dropna(subset=['order_delivered_customer_date'],inplace=True)

In [343]:
#결측치 제거 완료
df.isnull().sum()

customer_id                      0
customer_unique_id               0
customer_zip_code_prefix         0
order_id                         0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_customer_date    0
order_estimated_delivery_date    0
total_orders                     0
product_id                       0
seller_id                        0
price                            0
freight_value                    0
total_order_items                0
payment_sequential               0
payment_type                     0
payment_installments             0
payment_value                    0
product_category_name            0
product_weight_g                 0
product_length_cm                0
product_height_cm                0
product_width_cm                 0
product_category_name_english    0
is_once                          0
is_delayed                       0
duration_purchase_estimated      0
duration_purchase_ap

In [344]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115034 entries, 0 to 118433
Data columns (total 32 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   customer_id                    115034 non-null  object        
 1   customer_unique_id             115034 non-null  object        
 2   customer_zip_code_prefix       115034 non-null  int64         
 3   order_id                       115034 non-null  object        
 4   order_status                   115034 non-null  object        
 5   order_purchase_timestamp       115034 non-null  datetime64[ns]
 6   order_approved_at              115034 non-null  datetime64[ns]
 7   order_delivered_customer_date  115034 non-null  datetime64[ns]
 8   order_estimated_delivery_date  115034 non-null  datetime64[ns]
 9   total_orders                   115034 non-null  int64         
 10  product_id                     115034 non-null  object        
 11  selle

In [345]:
df.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'order_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'total_orders', 'product_id',
       'seller_id', 'price', 'freight_value', 'total_order_items',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'product_category_name', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'product_category_name_english', 'is_once', 'is_delayed',
       'duration_purchase_estimated', 'duration_purchase_approved',
       'duration_purchase_delivered', 'duration_approved_delivered',
       'duration_approved_estimated'],
      dtype='object')

In [346]:
#필요없는 명목형 열 제거
df.drop(['customer_id','customer_unique_id','order_id','product_id','product_category_name','seller_id'], axis=1, inplace=True)

In [347]:
df.columns

Index(['customer_zip_code_prefix', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'total_orders', 'price',
       'freight_value', 'total_order_items', 'payment_sequential',
       'payment_type', 'payment_installments', 'payment_value',
       'product_weight_g', 'product_length_cm', 'product_height_cm',
       'product_width_cm', 'product_category_name_english', 'is_once',
       'is_delayed', 'duration_purchase_estimated',
       'duration_purchase_approved', 'duration_purchase_delivered',
       'duration_approved_delivered', 'duration_approved_estimated'],
      dtype='object')

In [348]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115034 entries, 0 to 118433
Data columns (total 26 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   customer_zip_code_prefix       115034 non-null  int64         
 1   order_status                   115034 non-null  object        
 2   order_purchase_timestamp       115034 non-null  datetime64[ns]
 3   order_approved_at              115034 non-null  datetime64[ns]
 4   order_delivered_customer_date  115034 non-null  datetime64[ns]
 5   order_estimated_delivery_date  115034 non-null  datetime64[ns]
 6   total_orders                   115034 non-null  int64         
 7   price                          115034 non-null  float64       
 8   freight_value                  115034 non-null  float64       
 9   total_order_items              115034 non-null  int64         
 10  payment_sequential             115034 non-null  float64       
 11  payme

In [349]:
# datetime 열 지우기
df.drop(['order_purchase_timestamp','order_approved_at','order_delivered_customer_date','order_estimated_delivery_date'], axis=1, inplace=True)

In [350]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115034 entries, 0 to 118433
Data columns (total 22 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   customer_zip_code_prefix       115034 non-null  int64  
 1   order_status                   115034 non-null  object 
 2   total_orders                   115034 non-null  int64  
 3   price                          115034 non-null  float64
 4   freight_value                  115034 non-null  float64
 5   total_order_items              115034 non-null  int64  
 6   payment_sequential             115034 non-null  float64
 7   payment_type                   115034 non-null  object 
 8   payment_installments           115034 non-null  float64
 9   payment_value                  115034 non-null  float64
 10  product_weight_g               115034 non-null  float64
 11  product_length_cm              115034 non-null  float64
 12  product_height_cm              1150

## 피처 엔지니어링(라벨링)

- order_status: 라벨링
- payment_type: 라벨링
- product_category_name_english: 라벨링

In [351]:
df['product_category_name_english'].value_counts()

product_category_name_english
bed_bath_table               11650
health_beauty                 9761
sports_leisure                8733
furniture_decor               8557
computers_accessories         7897
                             ...  
arts_and_craftmanship           24
la_cuisine                      16
cds_dvds_musicals               14
fashion_childrens_clothes        7
security_and_services            2
Name: count, Length: 72, dtype: int64

In [352]:
# 'product_category_name_english' 열의 값을 라벨링
labels, category_unique = pd.factorize(df['product_category_name_english'])

# 라벨된 데이터를 새 열로 추가
df['category_label'] = labels
df

,customer_zip_code_prefix,order_status,total_orders,price,freight_value,total_order_items,payment_sequential,payment_type,payment_installments,payment_value,...,product_width_cm,product_category_name_english,is_once,is_delayed,duration_purchase_estimated,duration_purchase_approved,duration_purchase_delivered,duration_approved_delivered,duration_approved_estimated,category_label
0,14409,delivered,1,124.99,21.88,1,1.0,credit_card,2.0,146.87,...,31.0,office_furniture,1,0,1673665.0,997.0,761400.0,760403.0,1672668.0,0
1,9790,delivered,1,289.00,46.48,1,1.0,credit_card,8.0,335.48,...,40.0,housewares,1,0,2085096.0,608.0,1439575.0,1438967.0,2084488.0,1
2,1151,delivered,1,139.94,17.79,1,1.0,credit_card,7.0,157.73,...,17.0,office_furniture,1,1,2101935.0,87085.0,2253066.0,2165981.0,2014850.0,0
3,8775,delivered,1,149.94,23.36,1,1.0,credit_card,1.0,173.30,...,28.0,office_furniture,1,0,2361202.0,4961.0,1295867.0,1290906.0,2356241.0,0
4,13056,delivered,1,230.00,22.25,1,1.0,credit_card,8.0,252.25,...,35.0,home_confort,1,0,1433310.0,1119.0,990258.0,989139.0,1432191.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,3937,delivered,1,74.90,13.88,1,1.0,credit_card,6.0,88.78,...,23.0,books_general_interest,1,0,1498303.0,1228.0,533900.0,532672.0,1497075.0,35
118430,6764,delivered,1,114.90,14.16,1,1.0,credit_card,3.0,129.06,...,22.0,sports_leisure,1,0,1352378.0,890.0,642863.0,641973.0,1351488.0,3
118431,60115,delivered,1,37.00,19.04,1,1.0,credit_card,5.0,56.04,...,18.0,health_beauty,1,1,2000890.0,1093.0,2674285.0,2673192.0,1999797.0,11
118432,92120,delivered,1,689.00,22.07,1,1.0,credit_card,2.0,711.07,...,14.0,watches_gifts,1,0,2688687.0,1367.0,1119006.0,1117639.0,2687320.0,16


In [353]:
labels, unique

(array([ 0,  1,  0, ..., 11, 16, 18], dtype=int64),
 Index(['credit_card', 'debit_card', 'voucher', 'boleto'], dtype='object'))

In [354]:
df.drop('product_category_name_english',axis=1,inplace=True)

In [355]:
df['order_status'].value_counts()

order_status
delivered    115027
canceled          7
Name: count, dtype: int64

In [356]:
# 'order_status' 열의 값을 라벨링
labels, unique = pd.factorize(df['order_status'])

# 라벨된 데이터를 새 열로 추가
df['order_status'] = labels
df

,customer_zip_code_prefix,order_status,total_orders,price,freight_value,total_order_items,payment_sequential,payment_type,payment_installments,payment_value,...,product_height_cm,product_width_cm,is_once,is_delayed,duration_purchase_estimated,duration_purchase_approved,duration_purchase_delivered,duration_approved_delivered,duration_approved_estimated,category_label
0,14409,0,1,124.99,21.88,1,1.0,credit_card,2.0,146.87,...,64.0,31.0,1,0,1673665.0,997.0,761400.0,760403.0,1672668.0,0
1,9790,0,1,289.00,46.48,1,1.0,credit_card,8.0,335.48,...,15.0,40.0,1,0,2085096.0,608.0,1439575.0,1438967.0,2084488.0,1
2,1151,0,1,139.94,17.79,1,1.0,credit_card,7.0,157.73,...,52.0,17.0,1,1,2101935.0,87085.0,2253066.0,2165981.0,2014850.0,0
3,8775,0,1,149.94,23.36,1,1.0,credit_card,1.0,173.30,...,51.0,28.0,1,0,2361202.0,4961.0,1295867.0,1290906.0,2356241.0,0
4,13056,0,1,230.00,22.25,1,1.0,credit_card,8.0,252.25,...,15.0,35.0,1,0,1433310.0,1119.0,990258.0,989139.0,1432191.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,3937,0,1,74.90,13.88,1,1.0,credit_card,6.0,88.78,...,22.0,23.0,1,0,1498303.0,1228.0,533900.0,532672.0,1497075.0,35
118430,6764,0,1,114.90,14.16,1,1.0,credit_card,3.0,129.06,...,24.0,22.0,1,0,1352378.0,890.0,642863.0,641973.0,1351488.0,3
118431,60115,0,1,37.00,19.04,1,1.0,credit_card,5.0,56.04,...,20.0,18.0,1,1,2000890.0,1093.0,2674285.0,2673192.0,1999797.0,11
118432,92120,0,1,689.00,22.07,1,1.0,credit_card,2.0,711.07,...,13.0,14.0,1,0,2688687.0,1367.0,1119006.0,1117639.0,2687320.0,16


In [357]:
df['payment_type'].value_counts()

payment_type
credit_card    84896
boleto         22362
voucher         6123
debit_card      1653
Name: count, dtype: int64

In [358]:
# 'order_status' 열의 값을 라벨링
labels, unique = pd.factorize(df['payment_type'])

# 라벨된 데이터를 새 열로 추가
df['payment_type'] = labels
df

,customer_zip_code_prefix,order_status,total_orders,price,freight_value,total_order_items,payment_sequential,payment_type,payment_installments,payment_value,...,product_height_cm,product_width_cm,is_once,is_delayed,duration_purchase_estimated,duration_purchase_approved,duration_purchase_delivered,duration_approved_delivered,duration_approved_estimated,category_label
0,14409,0,1,124.99,21.88,1,1.0,0,2.0,146.87,...,64.0,31.0,1,0,1673665.0,997.0,761400.0,760403.0,1672668.0,0
1,9790,0,1,289.00,46.48,1,1.0,0,8.0,335.48,...,15.0,40.0,1,0,2085096.0,608.0,1439575.0,1438967.0,2084488.0,1
2,1151,0,1,139.94,17.79,1,1.0,0,7.0,157.73,...,52.0,17.0,1,1,2101935.0,87085.0,2253066.0,2165981.0,2014850.0,0
3,8775,0,1,149.94,23.36,1,1.0,0,1.0,173.30,...,51.0,28.0,1,0,2361202.0,4961.0,1295867.0,1290906.0,2356241.0,0
4,13056,0,1,230.00,22.25,1,1.0,0,8.0,252.25,...,15.0,35.0,1,0,1433310.0,1119.0,990258.0,989139.0,1432191.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,3937,0,1,74.90,13.88,1,1.0,0,6.0,88.78,...,22.0,23.0,1,0,1498303.0,1228.0,533900.0,532672.0,1497075.0,35
118430,6764,0,1,114.90,14.16,1,1.0,0,3.0,129.06,...,24.0,22.0,1,0,1352378.0,890.0,642863.0,641973.0,1351488.0,3
118431,60115,0,1,37.00,19.04,1,1.0,0,5.0,56.04,...,20.0,18.0,1,1,2000890.0,1093.0,2674285.0,2673192.0,1999797.0,11
118432,92120,0,1,689.00,22.07,1,1.0,0,2.0,711.07,...,13.0,14.0,1,0,2688687.0,1367.0,1119006.0,1117639.0,2687320.0,16


In [359]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115034 entries, 0 to 118433
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   customer_zip_code_prefix     115034 non-null  int64  
 1   order_status                 115034 non-null  int64  
 2   total_orders                 115034 non-null  int64  
 3   price                        115034 non-null  float64
 4   freight_value                115034 non-null  float64
 5   total_order_items            115034 non-null  int64  
 6   payment_sequential           115034 non-null  float64
 7   payment_type                 115034 non-null  int64  
 8   payment_installments         115034 non-null  float64
 9   payment_value                115034 non-null  float64
 10  product_weight_g             115034 non-null  float64
 11  product_length_cm            115034 non-null  float64
 12  product_height_cm            115034 non-null  float64
 13  prod

# 이진분류 모델

In [360]:
category_unique

Index(['office_furniture', 'housewares', 'home_confort', 'sports_leisure',
       'computers_accessories', 'unknown', 'toys', 'furniture_decor', 'auto',
       'air_conditioning', 'telephony', 'health_beauty', 'garden_tools',
       'pet_shop', 'bed_bath_table', 'baby', 'watches_gifts',
       'kitchen_dining_laundry_garden_furniture', 'perfumery', 'art',
       'stationery', 'fashio_female_clothing', 'consoles_games',
       'construction_tools_lights', 'food_drink', 'drinks', 'cool_stuff',
       'fashion_bags_accessories', 'home_construction', 'luggage_accessories',
       'electronics', 'home_appliances_2', 'fashion_male_clothing',
       'small_appliances', 'small_appliances_home_oven_and_coffee',
       'books_general_interest', 'home_appliances', 'costruction_tools_tools',
       'signaling_and_security', 'musical_instruments',
       'construction_tools_construction', 'music', 'fashion_shoes',
       'industry_commerce_and_business', 'fashion_underwear_beach',
       'dvds_blu_

In [378]:
# predict_once 함수 정의
def predict_once(model, scaler, category_unique, customer_zip_code_prefix, order_status, total_orders, price, freight_value, 
                     total_order_items, payment_sequential, payment_type, payment_installments, payment_value, 
                     product_weight_g, product_length_cm, product_height_cm, product_width_cm,
                    is_delayed, duration_purchase_estimated, duration_purchase_approved, duration_purchase_delivered,
                    duration_approved_delivered, duration_approved_estimated, category_label):
    
    # category_label이 문자열로 입력되었을 경우의 처리
    if isinstance(category_label, str):
        category_label = unique.get_loc(category_label)
        
    # 입력된 데이터를 데이터프레임으로 변환합니다.
    # 위에서 피처엔지니어링한 방식대로(모델의 input으로 적절하게) input 값을 변환해줍니다. 
    input_data = pd.DataFrame({
        'customer_zip_code_prefix': [customer_zip_code_prefix],
        'order_status' :[0 if order_status == 'delivered' else 1],
        'total_orders': [total_orders], 
        'price': [price], 
        'freight_value': [freight_value],
        'total_order_items': [total_order_items],
        'payment_sequential': [payment_sequential], 
        'payment_type': [0 if payment_type == 'credit_card' else (1 if payment_type == 'boleto' else (2 if payment_type == 'voucher' else 3))],
        'payment_installments': [payment_installments],
        'payment_value' : [payment_value],
        'product_weight_g': [product_weight_g],
        'product_length_cm': [product_length_cm],
        'product_height_cm': [product_height_cm],
        'product_width_cm': [product_width_cm],
        'is_delayed': [is_delayed],
        'duration_purchase_estimated': [duration_purchase_estimated],
        'duration_purchase_approved': [duration_purchase_approved],
        'duration_purchase_delivered': [duration_purchase_delivered],
        'duration_approved_delivered':[duration_approved_delivered],
        'duration_approved_estimated':[duration_approved_estimated],
        'category_label':[category_label],
    })

    # 입력 데이터를 스케일링합니다.
    input_data_scaled = scaler.transform(input_data)
    
    # 예측을 수행합니다.
    prediction = model.predict(input_data_scaled)
    prediction_proba = model.predict_proba(input_data_scaled)

    # 예측 결과를 반환합니다.
    result = "once" if prediction == 1 else "not once"
    probability = prediction_proba[0][int(prediction)]  # 예측된 클래스의 확률을 반환합니다.
    
    return result, probability

# target, feature 분리

In [362]:
X = df.drop('is_once',axis=1)
X

,customer_zip_code_prefix,order_status,total_orders,price,freight_value,total_order_items,payment_sequential,payment_type,payment_installments,payment_value,...,product_length_cm,product_height_cm,product_width_cm,is_delayed,duration_purchase_estimated,duration_purchase_approved,duration_purchase_delivered,duration_approved_delivered,duration_approved_estimated,category_label
0,14409,0,1,124.99,21.88,1,1.0,0,2.0,146.87,...,54.0,64.0,31.0,0,1673665.0,997.0,761400.0,760403.0,1672668.0,0
1,9790,0,1,289.00,46.48,1,1.0,0,8.0,335.48,...,89.0,15.0,40.0,0,2085096.0,608.0,1439575.0,1438967.0,2084488.0,1
2,1151,0,1,139.94,17.79,1,1.0,0,7.0,157.73,...,52.0,52.0,17.0,1,2101935.0,87085.0,2253066.0,2165981.0,2014850.0,0
3,8775,0,1,149.94,23.36,1,1.0,0,1.0,173.30,...,56.0,51.0,28.0,0,2361202.0,4961.0,1295867.0,1290906.0,2356241.0,0
4,13056,0,1,230.00,22.25,1,1.0,0,8.0,252.25,...,45.0,15.0,35.0,0,1433310.0,1119.0,990258.0,989139.0,1432191.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,3937,0,1,74.90,13.88,1,1.0,0,6.0,88.78,...,22.0,22.0,23.0,0,1498303.0,1228.0,533900.0,532672.0,1497075.0,35
118430,6764,0,1,114.90,14.16,1,1.0,0,3.0,129.06,...,25.0,24.0,22.0,0,1352378.0,890.0,642863.0,641973.0,1351488.0,3
118431,60115,0,1,37.00,19.04,1,1.0,0,5.0,56.04,...,25.0,20.0,18.0,1,2000890.0,1093.0,2674285.0,2673192.0,1999797.0,11
118432,92120,0,1,689.00,22.07,1,1.0,0,2.0,711.07,...,19.0,13.0,14.0,0,2688687.0,1367.0,1119006.0,1117639.0,2687320.0,16


In [363]:
y = df['is_once']
y

0         1
1         1
2         1
3         1
4         1
         ..
118429    1
118430    1
118431    1
118432    1
118433    1
Name: is_once, Length: 115034, dtype: int64

# 데이터 셋을 훈련 세트와 테스트 세트로 나누기

In [364]:
# sklearn.model_selection의 train_test_split 함수를 import 해주세요.
from sklearn.model_selection import train_test_split

# tran_test_split 함수를 이용하여 독립변수(X)와 종속변수(y)를 각각 훈련 세트와 테스트 세트로 나누어주세요. 각 훈련 세트와 테스트 세트의 변수 명은 아래와 같습니다.
# 독립변수 훈련 세트: X_train, 독립변수 테스트 세트: X_test, 종속변수 훈련 세트: y_train, 종속변수 테스트 세트: y_test
# 단, random_state는 42로 합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =42)

In [365]:
print("len(X_train): ",len(X_train))
print("len(X_test): ",len(X_test))
print("len(X_test)/(len(X_train)+len(X_test))",len(X_test)/(len(X_train)+len(X_test)))

len(X_train):  92027
len(X_test):  23007
len(X_test)/(len(X_train)+len(X_test)) 0.20000173861640907


# feature에 scaler 적용

In [366]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 92027 entries, 103923 to 16266
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   customer_zip_code_prefix     92027 non-null  int64  
 1   order_status                 92027 non-null  int64  
 2   total_orders                 92027 non-null  int64  
 3   price                        92027 non-null  float64
 4   freight_value                92027 non-null  float64
 5   total_order_items            92027 non-null  int64  
 6   payment_sequential           92027 non-null  float64
 7   payment_type                 92027 non-null  int64  
 8   payment_installments         92027 non-null  float64
 9   payment_value                92027 non-null  float64
 10  product_weight_g             92027 non-null  float64
 11  product_length_cm            92027 non-null  float64
 12  product_height_cm            92027 non-null  float64
 13  product_width_cm

In [367]:
# sklearn.preprocessing에서 MinMaxScaler를 import 해옵니다.
from sklearn.preprocessing import MinMaxScaler

# scaler라는 변수를 MinMaxScaler 함수로 선언합니다.
scaler = MinMaxScaler()

# X_train_scaled라는 변수에 스케일링된 독립변수 훈련 세트를 저장합니다.
X_train_scaled = scaler.fit_transform(X_train)

# X_test_scaled라는 변수에 스케일링된 독립변수 테스트 세트를 저장합니다.
X_test_scaled = scaler.transform(X_test)

In [368]:
# NumPy 배열을 Pandas DataFrame으로 변환
X_train_scaled_df = pd.DataFrame(X_train_scaled)

# 이제 isnull().sum()을 사용할 수 있음
null_counts = X_train_scaled_df.isnull().sum()
X_train_scaled_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.194257,0.0,0.0,0.013684,0.053041,0.000000,0.0,0.0,0.083333,0.008396,...,0.380952,0.047619,0.372881,0.0,0.015703,0.000334,0.078585,0.111117,0.020131,0.098592
1,0.129353,0.0,0.0,0.048878,0.024580,0.000000,0.0,0.0,0.083333,0.024888,...,0.323810,0.038095,0.186441,0.0,0.006792,0.050686,0.025268,0.052163,0.010451,0.154930
2,0.319650,0.0,0.0,0.003349,0.036858,0.000000,0.0,0.0,0.083333,0.002818,...,0.190476,0.133333,0.169492,0.0,0.011935,0.000432,0.025028,0.059135,0.016377,0.042254
3,0.298726,0.0,0.0,0.001789,0.036858,0.000000,0.0,0.0,0.041667,0.002049,...,0.380952,0.038095,0.254237,0.0,0.013059,0.000279,0.087614,0.119885,0.017500,0.197183
4,0.380866,0.0,0.0,0.011754,0.063074,0.000000,0.0,0.0,0.041667,0.007746,...,0.180952,0.047619,0.110169,0.0,0.012150,0.000227,0.089654,0.121872,0.016596,0.140845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92022,0.398183,0.0,0.0,0.010254,0.057020,0.043478,0.0,0.0,0.041667,0.013650,...,0.190476,0.190476,0.169492,0.0,0.012740,0.002010,0.094637,0.126451,0.017154,0.042254
92023,0.891439,0.0,0.0,0.007150,0.036858,0.000000,0.0,0.0,0.250000,0.004691,...,0.171429,0.028571,0.237288,0.0,0.011473,0.001535,0.049585,0.082805,0.015900,0.140845
92024,0.368166,0.0,0.0,0.006690,0.035442,0.043478,0.0,0.0,0.125000,0.008844,...,0.285714,0.028571,0.169492,0.0,0.022510,0.000235,0.072736,0.105455,0.026910,0.098592
92025,0.126110,0.0,0.0,0.031057,0.090266,0.000000,0.0,0.0,0.166667,0.018074,...,0.600000,0.590476,0.203390,0.0,0.012230,0.000223,0.079989,0.112494,0.016675,0.000000


# 로지스틱 회귀모형

In [369]:
# sklearn.linear_model에서 LogisticRegression 함수를 import 해오세요.
from sklearn.linear_model import LogisticRegression

# lr_model을 변수명으로 해서 로지스틱회귀 모델 객체를 생성하세요.
lr_model = LogisticRegression()

In [370]:
# lr_model 모델을 훈련 세트로 학습시키세요.
lr_model.fit(X_train_scaled, y_train)

LogisticRegression()

In [371]:
y_pred = lr_model.predict(X_test_scaled)

In [372]:
# sklearn.metrics에서 confusion_matrix를 import 해옵니다.
from sklearn.metrics import confusion_matrix

# 실제 타겟(y_test)과 예측 타겟(y_pred)을 사용하여 혼동 행렬 생성해서, conf_matrix 변수에 저장해주세요.
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

array([[ 1538,     0],
       [   10, 21459]], dtype=int64)

In [373]:
print(conf_matrix)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(f'True Negatives: {tn}')
print(f'False Positives: {fp}')
print(f'False Negatives: {fn}')
print(f'True Positives: {tp}')

[[ 1538     0]
 [   10 21459]]
True Negatives: 1538
False Positives: 0
False Negatives: 10
True Positives: 21459


In [374]:
# 정확도
# sklearn.metrics에서 accuracy_score를 import 해오세요.
from sklearn.metrics import accuracy_score

# accuracy 변수에 정확도를 저장해주세요.
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9995653496761855

In [375]:
print(f'로지스틱회귀 모델의 정확도: {accuracy}')

로지스틱회귀 모델의 정확도: 0.9995653496761855


In [376]:
# sklearn.metrics에서 classification_report를 import 해오세요.
from sklearn.metrics import classification_report

# report 변수에 분류 보고서를 저장하세요.
report = classification_report(y_test, y_pred)

In [377]:
print(f'[ 분류 보고서 ]\n {report}')

[ 분류 보고서 ]
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      1538
           1       1.00      1.00      1.00     21469

    accuracy                           1.00     23007
   macro avg       1.00      1.00      1.00     23007
weighted avg       1.00      1.00      1.00     23007



# 의사결정나무

In [379]:
# sklearn.tree에서 DecisionTreeClassifier를 import 해오세요.
from sklearn.tree import DecisionTreeClassifier

# tree_model이라는 변수에 의사 결정 나무 모델을 생성해주세요. 단, random_state는 42로 합니다.
tree_model = DecisionTreeClassifier(random_state=42)

In [380]:
df.describe()

,customer_zip_code_prefix,order_status,total_orders,price,freight_value,total_order_items,payment_sequential,payment_type,payment_installments,payment_value,...,product_height_cm,product_width_cm,is_once,is_delayed,duration_purchase_estimated,duration_purchase_approved,duration_purchase_delivered,duration_approved_delivered,duration_approved_estimated,category_label
count,115034.000000,115034.000000,115034.000000,115034.000000,115034.000000,115034.000000,115034.000000,115034.000000,115034.000000,115034.000000,...,115034.000000,115034.000000,115034.000000,115034.000000,1.150340e+05,1.150340e+05,1.150340e+05,1.150340e+05,1.150340e+05,115034.000000
mean,35081.536198,0.000061,1.085157,120.079504,19.995506,1.507085,1.090686,0.704009,2.936688,171.911525,...,16.588174,23.055071,0.931542,0.078446,2.063057e+06,3.755256e+04,1.078596e+06,1.041043e+06,2.025505e+06,13.928491
std,29851.384178,0.007801,0.399179,183.029790,15.743494,1.327824,0.684363,1.217053,2.772367,266.109922,...,13.428016,11.734699,0.252531,0.268874,1.124161e+06,7.537328e+04,8.167409e+05,8.138022e+05,1.124409e+06,11.772845
min,1003.000000,0.000000,1.000000,0.850000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.734920e+05,0.000000e+00,4.608700e+04,-6.039060e+05,-5.757890e+05,0.000000
25%,11310.000000,0.000000,1.000000,39.900000,13.080000,1.000000,1.000000,0.000000,1.000000,60.850000,...,8.000000,15.000000,1.000000,0.000000,1.589377e+06,7.770000e+02,5.830822e+05,5.439370e+05,1.559137e+06,6.000000
50%,24320.000000,0.000000,1.000000,74.900000,16.280000,1.000000,1.000000,0.000000,2.000000,108.190000,...,13.000000,20.000000,1.000000,0.000000,2.008822e+06,1.248000e+03,8.810170e+05,8.487995e+05,1.969565e+06,11.000000
75%,58801.750000,0.000000,1.000000,133.000000,21.180000,2.000000,1.000000,1.000000,4.000000,188.940000,...,20.000000,30.000000,1.000000,0.000000,2.459483e+06,5.370475e+04,1.342800e+06,1.299688e+06,2.430518e+06,16.000000
max,99980.000000,1.000000,17.000000,6735.000000,409.680000,24.000000,26.000000,3.000000,24.000000,13664.080000,...,105.000000,118.000000,1.000000,1.000000,1.659439e+08,2.669197e+06,1.811191e+07,1.801448e+07,1.659175e+08,71.000000


In [381]:
# 모델 학습
tree_model.fit(X_train_scaled, y_train)

# 예측 결과 생성
tree_pred = tree_model.predict(X_test_scaled)

# 혼동 행렬 생성
tn, fp, fn, tp = confusion_matrix(y_test, tree_pred).ravel()
print(f'True Negatives: {tn}')
print(f'False Positives: {fp}')
print(f'False Negatives: {fn}')
print(f'True Positives: {tp}')

# 정확도 측정
tree_accuracy = accuracy_score(y_test, tree_pred)
print("Decision Tree 모델의 정확도:", tree_accuracy)

# 분류 보고서 생성
tree_report = classification_report(y_test, tree_pred)
print(tree_report)


True Negatives: 1538
False Positives: 0
False Negatives: 0
True Positives: 21469
Decision Tree 모델의 정확도: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1538
           1       1.00      1.00      1.00     21469

    accuracy                           1.00     23007
   macro avg       1.00      1.00      1.00     23007
weighted avg       1.00      1.00      1.00     23007



# SVM

In [382]:
# sklearn.svm에서 SVC를 import 해오세요.
from sklearn.svm import SVC

# svm_model이라는 변수에 서포트 벡터 머신 모델을 생성해주세요. 단, random_state는 42, probability=True로 합니다.
svm_model = SVC(random_state = 42, probability=True)

In [383]:
# 모델 훈련
svm_model.fit(X_train_scaled, y_train)

# SVM 예측 결과 생성
svm_pred = svm_model.predict(X_test_scaled)

# 혼동 행렬 생성
tn, fp, fn, tp = confusion_matrix(y_test, svm_pred).ravel()
print(f'True Negatives: {tn}')
print(f'False Positives: {fp}')
print(f'False Negatives: {fn}')
print(f'True Positives: {tp}')

# 정확도 측정
svm_accuracy = accuracy_score(y_test, svm_pred)
print("SVM 모델의 정확도:", svm_accuracy)

# 분류 보고서 생성
svm_report = classification_report(y_test, svm_pred)
print(svm_report)


True Negatives: 1532
False Positives: 6
False Negatives: 0
True Positives: 21469
SVM 모델의 정확도: 0.9997392098057113
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1538
           1       1.00      1.00      1.00     21469

    accuracy                           1.00     23007
   macro avg       1.00      1.00      1.00     23007
weighted avg       1.00      1.00      1.00     23007



# kNN

In [385]:
# sklearn.neighbors에서 KNeighborsClassifier를 import 해오세요.
from  sklearn.neighbors import KNeighborsClassifier

# knn_model이라는 변수에 kNN 모델을 생성해주세요. 단, n_neighbors=5로 합니다.
knn_model = KNeighborsClassifier(n_neighbors = 5)

In [386]:
# 모델 학습
knn_model.fit(X_train_scaled, y_train)

# 예측 수행
knn_pred = knn_model.predict(X_test_scaled)

# 혼동 행렬 생성
tn, fp, fn, tp = confusion_matrix(y_test, knn_pred).ravel()
print(f'True Negatives: {tn}')
print(f'False Positives: {fp}')
print(f'False Negatives: {fn}')
print(f'True Positives: {tp}')

# 정확도 측정
knn_accuracy = accuracy_score(y_test, knn_pred)
print("kNN 모델의 정확도 : ", knn_accuracy)

# 분류 보고서 생성
knn_report = classification_report(y_test, knn_pred)
print(knn_report)


True Negatives: 813
False Positives: 725
False Negatives: 16
True Positives: 21453
kNN 모델의 정확도 :  0.9677924110053462
              precision    recall  f1-score   support

           0       0.98      0.53      0.69      1538
           1       0.97      1.00      0.98     21469

    accuracy                           0.97     23007
   macro avg       0.97      0.76      0.83     23007
weighted avg       0.97      0.97      0.96     23007



In [262]:
result, probability = predict_survival(model=tree_model, scaler, 
                                       category_unique, customer_zip_code_prefix=40000, 
                                       order_status=0, total_orders=1, price=40, freight_value=10, 
                                       total_order_items=1, payment_sequential=1, payment_type='credit_card', 
                                       payment_installments=2, payment_value=40, 
                                       product_weight_g=20, product_length_cm=20, product_height_cm=20, product_width_cm=10,
                                       is_delayed=1, duration_purchase_estimated, duration_purchase_approved, 
                                       duration_purchase_delivered, duration_approved_delivered, 
                                       duration_approved_estimated, category_label='bed_bath_table')

NameError: name 'predict_survival' is not defined